In [1]:
!pip install -U sentence-transformers torch
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of th

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample
from sentence_transformers import losses
from torch.utils.data import DataLoader
from datasets import Dataset
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from google.colab import drive
import shutil

drive.mount('/content/gdrive')

source_path = '/content/gdrive/MyDrive/dataset_text_mining/pre_processed.csv'

destination_path = '/content/dataset_yelp.csv'

shutil.copyfile(source_path, destination_path)

Mounted at /content/gdrive


'/content/dataset_yelp.csv'

In [4]:
df = pd.read_csv('/content/dataset_yelp.csv')
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,...,Mexican,Beauty & Spas,Pubs,Cajun/Creole,Event Planning & Services,Venues & Event Spaces,Bakeries,Hotels,Specialty Food,Main_category
0,4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,...,0,0,0,0,0,0,0,0,0,Sandwiches
1,10,rGQRf8UafX7OTlMNN19I8A,1WHRWwQmZOZDAhp2Qyny4g,uMvVYRgGNXf5boolA9HXTw,5.0,2,0,0,My experience with Shalimar was nothing but wo...,2015-06-21 14:48:06,...,0,0,0,0,0,0,0,0,0,Shopping
2,12,XW_LfMv0fV21l9c6xQd_lw,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,0,0,0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,...,0,0,0,0,0,0,0,0,0,Bars
3,13,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14,...,0,0,0,0,0,0,0,0,0,Pizza
4,15,OAhBYw8IQ6wlfw1owXWRWw,1C2lxzUo1Hyye4RFIXly3g,BVndHaLihEYbr76Z0CMEGw,5.0,0,0,0,"Great place for breakfast! I had the waffle, w...",2014-10-11 16:22:06,...,0,0,0,0,0,0,0,0,0,Sandwiches


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
category_mapping = {'Bars':1, 'Nightlife':2, 'Event Planning & Services':3, 'Mexican':4, 'Breakfast & Brunch':5, 'Pizza':6, 'Sandwiches':7, 'Italian':8, 'Seafood':9,
                    'Coffee & Tea':10, 'Ice Cream & Frozen Yogurt':11, 'Beauty & Spas':12, 'Hotels & Travel':13, 'Shopping':14, 'Burgers':15, 'Sushi Bars':16, 'German':17,
                    'Thai':18, 'Cafes':19}

df["category_label"] = df["Main_category"].map(category_mapping)



# Primo fine tuning

In [7]:
text1 = []
text2 = []
for i in range(10000):
   random_rows = random.sample(df['text'].tolist(), 2)
   text1.append(random_rows[0])
   text2.append(random_rows[1])

df_train = pd.DataFrame({'text1': text1, 'text2': text2})

In [8]:
texts = df_train['text1'].tolist() + df_train['text2'].tolist()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
tfidf_text1 = tfidf_matrix[:len(df_train)]
tfidf_text2 = tfidf_matrix[len(df_train):]

df_train['similarity'] = [
    cosine_similarity(tfidf_text1[i], tfidf_text2[i])[0, 0]
    for i in range(len(df_train))
]

In [9]:
train_examples = [
    InputExample(texts=[row["text1"], row["text2"]], label=row["similarity"])
    for _, row in df_train.iterrows()
]

# Secondo fine tuning

In [10]:
df_tmp = df.sample(n = 40000)

train_examples = []
for i in range(len(df_tmp)):
  row = df_tmp.iloc[i]
  train_examples.append(InputExample(texts=[row['text']], label=row['category_label']))

In [15]:
print(train_examples[0])
print(train_examples[1])
print(train_examples[6])

<InputExample> label: 7, texts: I'm one of their regulars and the food is pretty consistent. Their BBQ chicken pizza with thin crust and Chicken Alfredo pasta are amazing! We order from them weekly and delivery is pretty fast. I don't know why the reviews are so low because they never disappoint me.
<InputExample> label: 3, texts: We stayed here for a long weekend last year. 2 kids, 1 a toddler and we had a great time. All the restaurants were good. My son and husband had fun at the arcade. Hubs enjoyed gambling. My favorite hotel in Reno.
<InputExample> label: 7, texts: Never have left a restaurant so upset and disappointed in a experience. Took over an hour for food on a Monday night with a half empty restaurant. No apology no excuses, just a simple lack of caring for the service of their guests. Food was good, but it's not worth the long wait, lack of service, and utter disregard for guests. I'd take my business elsewhere. Their servers and  bartenders truly lack what the service in

# Fine tuning

In [ ]:
train_loss = losses.BatchAllTripletLoss(model)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    output_path='/content/fine_tuned_model1'
)

Step,Training Loss
500,4.432900
1000,4.421600
1500,4.404900
2000,4.398600
2500,4.390400
3000,4.375000
3500,4.371500
4000,4.357400
4500,4.355900
5000,4.336700


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]